In [11]:
import requests      # Библиотека для отправки запросов
import numpy as np   # Библиотека для матриц, векторов и линала
import pandas as pd  # Библиотека для табличек
import time          # Библиотека для тайм-менеджмента
from bs4 import BeautifulSoup

In [2]:
page_link = 'http://knowyourmeme.com/memes/all/page/1'
response = requests.get(page_link)
response

<Response [403]>

Попробуем выяснить, почему. Для этого проверим, как выглядел финальный запрос, отправленный нами на сервер, а конкретнее — как выглядел наш User-Agent в глазах сервера

In [4]:
for key,value in response.request.headers.items():
    print(f'Key: {key} Value: {value}')

Key: User-Agent Value: python-requests/2.24.0
Key: Accept-Encoding Value: gzip, deflate
Key: Accept Value: */*
Key: Connection Value: keep-alive


In [7]:
from fake_useragent import UserAgent
UserAgent().chrome

'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/33.0.1750.517 Safari/537.36'

Попробуем прогнать наш запрос еще раз, уже со сгенерированным агентом

In [9]:
response = requests.get(page_link, headers = {'User-Agent': UserAgent().chrome})
response

<Response [200]>

In [10]:
html = response.content
html[:1000]

b'<!DOCTYPE html>\n<html xmlns:fb=\'https://www.facebook.com/2008/fbml\' xmlns=\'https://www.w3.org/1999/xhtml\'>\n<head>\n<meta content=\'text/html; charset=utf-8\' http-equiv=\'Content-Type\'>\n<script type="text/javascript">window.NREUM||(NREUM={});NREUM.info={"beacon":"bam-cell.nr-data.net","errorBeacon":"bam-cell.nr-data.net","licenseKey":"c1a6d52f38","applicationID":"31165848","transactionName":"dFdfRUpeWglTQB8GDUNKWFRLHlcJWg==","queueTime":0,"applicationTime":54,"agent":""}</script>\n<script type="text/javascript">(window.NREUM||(NREUM={})).loader_config={licenseKey:"c1a6d52f38",applicationID:"31165848"};window.NREUM||(NREUM={}),__nr_require=function(e,t,n){function r(n){if(!t[n]){var i=t[n]={exports:{}};e[n][0].call(i.exports,function(t){var i=e[n][1][t];return r(i||t)},i,i.exports)}return t[n].exports}if("function"==typeof __nr_require)return __nr_require;for(var i=0;i<n.length;i++)r(n[i]);return r}({1:[function(e,t,n){function r(){}function i(e,t,n){return function(){return o

In [12]:
soup = BeautifulSoup(html,'html.parser')

In [13]:
soup

<!DOCTYPE html>

<html xmlns="https://www.w3.org/1999/xhtml" xmlns:fb="https://www.facebook.com/2008/fbml">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<script type="text/javascript">window.NREUM||(NREUM={});NREUM.info={"beacon":"bam-cell.nr-data.net","errorBeacon":"bam-cell.nr-data.net","licenseKey":"c1a6d52f38","applicationID":"31165848","transactionName":"dFdfRUpeWglTQB8GDUNKWFRLHlcJWg==","queueTime":0,"applicationTime":54,"agent":""}</script>
<script type="text/javascript">(window.NREUM||(NREUM={})).loader_config={licenseKey:"c1a6d52f38",applicationID:"31165848"};window.NREUM||(NREUM={}),__nr_require=function(e,t,n){function r(n){if(!t[n]){var i=t[n]={exports:{}};e[n][0].call(i.exports,function(t){var i=e[n][1][t];return r(i||t)},i,i.exports)}return t[n].exports}if("function"==typeof __nr_require)return __nr_require;for(var i=0;i<n.length;i++)r(n[i]);return r}({1:[function(e,t,n){function r(){}function i(e,t,n){return function(){return o(e,[u.now()].

In [14]:
soup.html.head.title

<title>All Entries | Know Your Meme</title>

Более того, зная адрес элемента, мы сразу можем найти его. Например, можно сделать это по классу. Следующая команда должна найти элемент, который лежит внутри тега a и имеет класс photo.

In [15]:
obj = soup.find(lambda tag: tag.name == 'a' and tag.get('class') == ['photo'])
obj

<a class="photo" href="/memes/nobody-asks-how-is-x"><img alt='Nobody Asks "How is X?"' data-src="https://i.kym-cdn.com/entries/icons/medium/000/037/240/howiswaldo16x9.jpg" src="https://s.kym-cdn.com/assets/blank-b3f96f160b75b1b49b426754ba188fe8.gif" title='Nobody Asks "How is X?"'/> <div class="entry-labels"> <span class="label label-submission"> Submission </span> </div> </a>

Полученный после поиска объект также обладает структурой bs4. Поэтому можно продолжить искать нужные нам объекты уже в нём! Вытащим ссылку на этот мем. Сделать это можно по атрибуту href, в котором лежит наша ссылка.

In [16]:
obj.attrs['href']

'/memes/nobody-asks-how-is-x'

Если несколько элементов на странице обладают указанным адресом, то метод find вернёт только самый первый. Чтобы найти все элементы с таким адресом, нужно использовать метод findAll, и на выход будет выдан список. Таким образом, мы можем получить одним поиском сразу все объекты, содержащие ссылки на страницы с мемами.

In [17]:
meme_links = soup.findAll(lambda tag: tag.name == 'a' and tag.get('class')==['photo'])
meme_links[:3]

[<a class="photo" href="/memes/nobody-asks-how-is-x"><img alt='Nobody Asks "How is X?"' data-src="https://i.kym-cdn.com/entries/icons/medium/000/037/240/howiswaldo16x9.jpg" src="https://s.kym-cdn.com/assets/blank-b3f96f160b75b1b49b426754ba188fe8.gif" title='Nobody Asks "How is X?"'/> <div class="entry-labels"> <span class="label label-submission"> Submission </span> </div> </a>,
 <a class="photo" href="/memes/kanyeiplier"><img alt="Kanye and markiplier surrounded by hearts referencing the meme." data-src="https://i.kym-cdn.com/entries/icons/medium/000/037/239/Kanyeiplier_Banner.jpg" src="https://s.kym-cdn.com/assets/blank-b3f96f160b75b1b49b426754ba188fe8.gif" title="#Kanyeiplier"/> <div class="entry-labels"> <span class="label label-submission"> Submission </span> </div> </a>,
 <a class="photo" href="/memes/doing-numbers"><img alt="Doing Numbers meme example showing a guy sarcastically reacting to a low-engagement tweet." data-src="https://i.kym-cdn.com/entries/icons/medium/000/037/238

Осталось очистить полученный список от мусора:

In [19]:
meme_links = [link.attrs['href'] for link in meme_links]
meme_links[:10]

['/memes/nobody-asks-how-is-x',
 '/memes/kanyeiplier',
 '/memes/doing-numbers',
 '/memes/wurmple',
 '/memes/youre-doin-good-lad',
 '/memes/events/2021-israel-palestine-crisis',
 '/memes/nobody-wants-to-work-anymore',
 '/memes/pain-hub',
 '/memes/antonymic-misinterpretation-x-implies-the-existence-of-y',
 '/memes/second-breakfast']

In [20]:
def getPageLinks(page_number):
    """
        Возвращает список ссылок на мемы, полученный с текущей страницы

        page_number: int/string
            номер страницы для парсинга

    """
    # составляем ссылку на страницу поиска
    page_link = 'http://knowyourmeme.com/memes/all/page/{}'.format(page_number)

    # запрашиваем данные по ней
    response = requests.get(page_link, headers={'User-Agent': UserAgent().chrome})

    if not response.ok:
        # если сервер нам отказал, вернем пустой лист для текущей страницы
        return []

    # получаем содержимое страницы и переводим в суп
    html = response.content
    soup = BeautifulSoup(html,'html.parser')

    # наконец, ищем ссылки на мемы и очищаем их от ненужных тэгов
    meme_links = soup.findAll(lambda tag: tag.name == 'a' and tag.get('class') == ['photo'])
    meme_links = ['http://knowyourmeme.com' + link.attrs['href'] for link in meme_links]

    return meme_links

In [21]:
meme_links = getPageLinks(1)
meme_links[:2]

['http://knowyourmeme.com/memes/nobody-asks-how-is-x',
 'http://knowyourmeme.com/memes/kanyeiplier']